#### Point Map Visualization

Point map visualization is natively supported, if your table outputs longitude-latitude pairs. It is recommended to limit the query to not too many rows for performance reasons.

In [0]:
%sql
select
  *
from
  centroids_lonlat
where
  -- filtering for gemeente to avoid too many rows to visualize
  gemeente_naam = 'Almere'

![point map](./img/point_map.png)